# Import Libraries

In [2]:
#imports Libs

import os
import pandas as pd
import boto3
import time
from botocore.client import ClientError

import pyarrow.parquet as pq
import s3fs
import calendar

import findspark                                              #Import library to Search for Spark Installation  

findspark.init()                                              #Search Spark Installation

import pyspark                                                #Only run after findspark.init()

from pyspark.sql import SparkSession                          #Import of Spark Session
from pyspark import SparkContext as spark                     #Import the Regular Spark Contex 
from pyspark.sql import SQLContext                            #Import the SQL Spark Contex
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext                                       #Initialize Spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1575983967584_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-------------

# Creating Variables

## Database

In [ ]:
ATHENA_SPG = os.environ['SPG_DATABASE']

## Buckets

In [ ]:
SPG_OUTPUT_BUCKET = os.environ['OUTPUT_BUCKET']

In [ ]:
SPG_QUERY_BUCKET = os.environ['QUERY_BUCKET']

## Input Paths

In [ ]:
SPG_QUERY_BUCKET_ATHENA = "SPG_QUERY/SPG_ATHENA"

In [ ]:
QUERY_AGENTS = "SELECT DISTINCT * FROM "+ATHENA_SPG+".tb_spg_pc_agents_gpd;"

In [ ]:
QUERY_COLLECT = "SELECT * FROM "+ATHENA_SPG+".tb_coleta;"

## Output Paths

In [ ]:
SPG_OUTPUT_BUCKET_RM = "SPG_WALLET/tb_SPG_WT_WALLET.parquet"

## Boto3 Variables

In [ ]:
#S3 Configuration
S3_ATHENA_INPUT =  's3://'+SPG_QUERY_BUCKET+'/'+SPG_QUERY_BUCKET_ATHENA

In [ ]:
S3_ATHENA_OUTPUT = 's3://'+SPG_QUERY_BUCKET+'/'+SPG_QUERY_BUCKET_ATHENA

In [ ]:
region_name = os.environ['AWS_REGION']

In [ ]:
aws_access_key_id = os.environ['AWS_ACCESS_KEY']

In [ ]:
aws_secret_access_key = os.environ['AWS_SECRET_KEY']

-------------

# Creating Defined Functions

In [6]:
# Run Query

def run_query(query, database, s3_output):
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
            },
        ResultConfiguration={
            'OutputLocation': s3_output,
            }
        )
    return response

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def get_aws_path(query,database,s3_output):
    response = run_query(query, database, s3_output)
    file_query = response['QueryExecutionId']
    file_metadata = response['QueryExecutionId'] + '.metadata'
    return file_query

In [ ]:
# Wating for 300 seconds until the end of the upload

def wait_athena_load(Bucket, Key):
    time_to_wait = 300
    time_counter = 0

    while True:
        try:
            s3.meta.client.head_object(Bucket=Bucket,Key=Key)
        except ClientError:
            time.sleep(1)
            time_counter += 1
            if time_counter > time_to_wait:
                break
        else:
            break

In [7]:
# Replacing Characters

def normalizing_date_hifen(col_name):
    removed_array = [('-', '/')]
    r = col_name
    for a, b in removed_array:
        r = regexp_replace(r, a, b)
    return r

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Replacing Characters

def normalizing_dots_coma(col_name):
    removed_chars = (".")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, ",")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-------------

# Configuring Boto3

In [ ]:
#Athena Client Configuration

client = boto3.client('athena', 
    aws_access_key_id = aws_access_key_id, 
    aws_secret_access_key = aws_secret_access_key, 
    region_name = region_name )

In [ ]:
#S3 Resource Configuration

s3 = boto3.resource('s3',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    region_name = region_name)

-------------

# Importing Tables

In [9]:
# Import CSV from View

athena_response = get_aws_path(QUERY_AGENTS,ATHENA_SPG,S3_ATHENA_OUTPUT)

wait_athena_load(SPG_QUERY_BUCKET, SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Import CSV from View

df_agents = spark.read.csv("s3a://"+SPG_QUERY_BUCKET+"/"+SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv", header = 'true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Importing Collect

In [12]:
# Import CSV from View

athena_response = get_aws_path(QUERY_COLLECT,ATHENA_SPG,S3_ATHENA_OUTPUT)

wait_athena_load(SPG_QUERY_BUCKET, SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Import CSV from View

df_collect = spark.read.csv("s3a://"+SPG_QUERY_BUCKET+"/"+SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv", header = 'true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

------------------

# Preparing Collect Table

In [15]:
df_collect=df_collect.withColumn("month", substring(df_collect["mesano"], 1,2))\
                   .withColumn("year", substring(df_collect["mesano"],4,8))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_collect=df_collect.withColumn("filial", when(df_collect.ovajustada.like("%CG%"), col("filial"))\
                                             .otherwise(concat(lit("Vendas "), col("estado"))))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_collect=df_collect.withColumn("soc_branch_month_year_approved", concat(col("ovajustada")
                                                                        ,lit("_")
                                                                        ,col("filial")
                                                                        ,lit("_")
                                                                        ,col("month")
                                                                        ,lit("_")
                                                                        ,col("year")
                                                                        ,lit("_")
                                                                        ,col("aprovado")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_collect=df_collect.withColumn("soc_branch_month_year_approved_gpd", concat(col("soc_branch_month_year_approved")
                                                                            ,lit("_")
                                                                            ,col("desc_gpd")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_collect=df_collect.groupBy("soc_branch_month_year_approved_gpd").agg(count("soc_branch_month_year_approved_gpd").alias("number_collect"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_collect.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[soc_branch_month_year_approved_gpd: string, number_collect: bigint]

---------

# Preparing Agents Table

In [21]:
df_agents=df_agents.withColumn("approved", lit("Sim"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_agents=df_agents.withColumn("month", substring(df_agents["mes-ano"], 1,2))\
                   .withColumn("year", substring(df_agents["mes-ano"],4,8))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_agents=df_agents.withColumn("soc_branch_month_year_approved", concat(col("ov_ajustada")
                                                                        ,lit("_")
                                                                        ,col("filial")
                                                                        ,lit("_")
                                                                        ,col("month")
                                                                        ,lit("_")
                                                                        ,col("year")
                                                                        ,lit("_")
                                                                        ,col("approved")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_agents=df_agents.withColumn("soc_branch_month_year_approved_gpd", concat(col("soc_branch_month_year_approved")
                                                                        ,lit("_")
                                                                        ,col("gpd_desc")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df_agents_grouped=df_agents.groupBy("soc_branch_month_year_approved_gpd").agg(sum("meta").alias("marks_grp"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df_agents=df_agents.join(df_agents_grouped, on=['soc_branch_month_year_approved_gpd'], how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df_agents=df_agents.join(df_collect, on=['soc_branch_month_year_approved_gpd'], how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df_agents=df_agents.withColumn("number_collect",when(col("number_collect").isNull(), lit(0)).otherwise(col("number_collect")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_agents=df_agents.drop("cod_vendedor"
                         ,"nome_vendedor"
                         ,"org_vendas"
                         ,"chavevendedor"
                         ,"unidade_de_analise"
                         ,"regiao_filial"
                         ,"meta"
                         ,"org_vendas").dropDuplicates(["soc_branch_month_year_approved_gpd"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df_agents.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[soc_branch_month_year_approved_gpd: string, ov_ajustada: string, filial: string, estado: string, gpd: string, gpd_desc: string, gpm: string, mes-ano: string, approved: string, month: string, year: string, soc_branch_month_year_approved: string, marks_grp: double, number_collect: bigint]

In [31]:
df_agents=df_agents.withColumn("marks_p100",normalizing_dots_coma((col("number_collect")/col("marks_grp")).cast("string")))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_agents=df_agents.withColumn("marks_p100",when(col("marks_p100").isNull(), "Sem Preço").otherwise(col("marks_p100")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_agents=df_agents.withColumn("adj_marks",(col("number_collect")/col("marks_grp")))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_agents=df_agents.withColumn("binary_marks",when(col("adj_marks").isNull(), lit(""))\
                                                  .otherwise(when(col("adj_marks")>=1, lit(1))\
                                                                 .otherwise(lit(0))))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_agents=df_agents.withColumn("marks_grp", col("marks_grp").cast("float"))\
                   .withColumn("adj_marks", col("adj_marks").cast("float"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_agents.write.partitionBy("gpd_desc")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_agents = df_agents.repartition("gpd_desc")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_agents.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[soc_branch_month_year_approved_gpd: string, ov_ajustada: string, filial: string, estado: string, gpd: string, gpd_desc: string, gpm: string, mes-ano: string, approved: string, month: string, year: string, soc_branch_month_year_approved: string, marks_grp: float, number_collect: bigint, marks_p100: string, adj_marks: float, binary_marks: string]

In [39]:
df_agents.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_RM, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…